# Lab 7 - PySpark i SQL, wiaderkowanie i partycjonowanie plików oraz zapi w hurtowni danych Hive.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

# ścieżka do bazy danych hurtowni danych oraz plików
# należy dostosować do ścieżki względnej, w której umieszczony został bieżący notebook
warehouse_location = '/opt/spark/work-dir/lab_07/metastore_db'

# utworzenie sesji Spark, ze wskazaniem włączenia obsługi Hive oraz
# lokalizacją przechowywania hurtowni danych
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Apache SQL and Hive")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .enableHiveSupport()\
        .config("spark.sql.warehouse.dir", warehouse_location)\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 12:07:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

## 1. Spark i SQL

Spark umożliwia zarejestrowanie obiektu DataFrame jako widoku, co umożliwia korzystanie z niego w sposób bardzo zbliżony do pracy z językiem SQL. Poniżej przykład.

In [167]:
# dostosuj ścieżkę do pliku do swoich danych, tutaj został utworzony mniejszy zbiór niż w poprzednim labie
df = spark.read.csv('../lab_06/employee_1m.csv', header=True, inferSchema=True)

In [168]:
# tworzymy widok tymczasowy w pamięci węzła
df.createOrReplaceTempView("EMPLOYEE_DATA")

In [60]:
# wypisanie tabeli, zwróć uwagę na to, czy stworzona tabela jest tymczasowa czy trwała
spark.catalog.listTables()

[Table(name='employee_bucketed', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='employees_partitioned_lastname', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='EMPLOYEE_DATA', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [169]:
# pobranie danych jak z tabeli SQL
spark.sql("Select * from EMPLOYEE_DATA limit 4").show()
spark.sql("select firstname from EMPLOYEE_DATA").show(10)

+---+---------+------------+---+-------+
| id|firstname|    lastname|age| salary|
+---+---------+------------+---+-------+
|  1|  Wisława|Mieczykowski| 24|6409.28|
|  2|    Agata|      Szczaw| 46|8980.03|
|  3|     Adam|Mieczykowski| 19|8817.88|
|  4|Krzysztof|Mieczykowski| 18|7531.72|
+---+---------+------------+---+-------+

+----------+
| firstname|
+----------+
|   Wisława|
|     Agata|
|      Adam|
| Krzysztof|
| Krzysztof|
|  Zbigniew|
| Krzysztof|
|      Adam|
|      Adam|
|Mieczysław|
+----------+
only showing top 10 rows



In [20]:
spark.sql("select firstname, count(firstname), avg(salary) from EMPLOYEE_DATA group by firstname").show()

+----------+----------------+------------------+
| firstname|count(firstname)|       avg(salary)|
+----------+----------------+------------------+
|   Wisława|           99640| 7848.952735246865|
|Mieczysław|          100565| 7851.463656441106|
|     Agata|          100127| 7853.484773637447|
| Krzysztof|           99704|7849.4506867327345|
|     Marek|           99741| 7846.229979948024|
|      Adam|          100170| 7847.951781371652|
| Katarzyna|          100650| 7853.827649180384|
|  Wojciech|           99553| 7856.233001315921|
|  Zbigniew|           99699|  7850.48046128846|
|Aleksandra|          100151| 7856.509769248448|
+----------+----------------+------------------+



In [28]:
rising = 0.1 # 10% podwyżki
spark.sql(f"select firstname, lastname, salary, round(salary + salary * {rising},2) as after_rising from EMPLOYEE_DATA").show(5)

+---------+------------+-------+------------+
|firstname|    lastname| salary|after_rising|
+---------+------------+-------+------------+
|  Wisława|Mieczykowski|6409.28|     7050.21|
|    Agata|      Szczaw|8980.03|     9878.03|
|     Adam|Mieczykowski|8817.88|     9699.67|
|Krzysztof|Mieczykowski|7531.72|     8284.89|
|Krzysztof|  Wróblewski|8371.61|     9208.77|
+---------+------------+-------+------------+
only showing top 5 rows



## 2. Apache Hive

https://hive.apache.org/


Apache Hive, który pierwotnie został stworzony w 2007 przez Facebooka, a następnie w 2008 przekazany pod skrzydła Apache Foundation, jest nazywany hurtownią danych. Dane przechowywane są głównie w systemie **HDFS** (**Hadoop Distributed File System**), ale Hive integruje się również z innymi silnikami baz danych.

Dostęp do danych jest realizowany przez **Hive QL**, który bardzo przypomina język SQL i taki sposób obsługi różnorodnych danych był jedną z głównych motywacji powstania Hive.

Za pomocą zapytań Hive QL (HQL) można wykonać takie zapytania jak:
* tworzenie i zmiana struktur tabel,
* import i export danych,
* agregacja danych, filtrowanie i złączenia danych.

Apache Hive jest wykorzystywany w dużych ekosystemach i mimo wymienionych wyżej zalet posiada również kilka ograniczeń:
* opóźnienie w czasie przetwarzania ze zwględu na wsadową naturę przetwarzania,
* brak możliwości przetwarzania real-time,
* język HQL nie daje możliwości wykonania takich operacji jak modyfikacja danych na poziomie wiersza,
* brak możliwości przeprowadzenia zaawansowanych analiz jak współczesne nowoczesne bazy SQL.

Alternatywne technologie:

* Presto
* Snowflake
* Apache Impala
* IBM Db2
* Google BigQuery
* Amazon Redshift
* ClickHouse
* Apache Hadoop
* Apache HBase
* Oracle Exadata
* Teradata Vantage
* Cloudera Impala

### 2.1 Hive QL

> Dokumentacja Apache Hive QL (dość archaiczna) jest dostępna pod adresem: https://cwiki.apache.org/confluence/display/Hive/LanguageManual

In [170]:
spark.catalog.currentCatalog()

'spark_catalog'

In [171]:
# dla zrealizowania kolejnych przykładów dokonamy kilku modyfikacji pliku employee
# 1. dodanie kolumny ID - indeksu
from pyspark.sql.functions import monotonically_increasing_id

df = df.withColumn("ID", monotonically_increasing_id())

In [172]:
df.show(10)

+---+----------+-------------------+---+--------+
| ID| firstname|           lastname|age|  salary|
+---+----------+-------------------+---+--------+
|  0|   Wisława|       Mieczykowski| 24| 6409.28|
|  1|     Agata|             Szczaw| 46| 8980.03|
|  2|      Adam|       Mieczykowski| 19| 8817.88|
|  3| Krzysztof|       Mieczykowski| 18| 7531.72|
|  4| Krzysztof|         Wróblewski| 31| 8371.61|
|  5|  Zbigniew|Brzęczyszczykiewicz| 30| 7884.59|
|  6| Krzysztof|       Mieczykowski| 21| 7110.25|
|  7|      Adam|       Mieczykowski| 50| 6661.27|
|  8|      Adam|             Szczaw| 41| 8383.29|
|  9|Mieczysław|         Malinowski| 37|10625.15|
+---+----------+-------------------+---+--------+
only showing top 10 rows



In [173]:
# dokonamy podziału danych i zapisania w różnych formatach
splits = df.randomSplit(weights=[0.3, 0.7], seed=19)

In [174]:
splits[0].count(), splits[1].count()

(298932, 701068)

In [ ]:
# to dość dziwne zjawisko niezbyt równego podziału danych jest opisane w artykułach:
# https://medium.com/udemy-engineering/pyspark-under-the-hood-randomsplit-and-sample-inconsistencies-examined-7c6ec62644bc
# oraz
# https://www.geeksforgeeks.org/pyspark-randomsplit-and-sample-methods/

In [175]:
# większa część trafi do nowej tymczasowej tabeli
splits[1].createOrReplaceTempView("EMPLOYEE_DATA_SPLIT_1")

In [176]:
# a mniejsza do plików JSON
splits[0].write.json('employee_data.json', mode='overwrite')

In [177]:
!ls ./employee_data.json/*.json

./employee_data.json/part-00000-c876c302-7856-404f-a1b9-e2a0a6f918b6-c000.json
./employee_data.json/part-00001-c876c302-7856-404f-a1b9-e2a0a6f918b6-c000.json


In [178]:
# aby móc wykorzystać dane w przykładach ze złączaniem, zapiszemy jeszcze próbkę danych z głównej ramki
# z identyfikatorami oraz dodatkową kolumną z podwyżką
from pyspark.sql.functions import col, lit, round

lucky_guys = spark.sql("select * from EMPLOYEE_DATA").sample(0.01)\
.withColumn('rising', lit('10%')).withColumn('salary after rising', round(col('salary') * 1.1, 2))

In [186]:
# zapisujemy szczęściarzy do oddzielnej tabeli w hurtowni
lucky_guys.write.mode('overwrite').saveAsTable("lucky_employees", format='parquet')

In [180]:
spark.catalog.listTables()

[Table(name='employee_firstname_bucketed', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='employee_id_bucketed', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='employee_salary_bucketed', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='employees_partitioned_lastname', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='lucky_eployees', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='EMPLOYEE_DATA', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='EMPLOYEE_DATA_SPLIT_1', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

#### Złączenie danych z różnych źródeł danych

In [188]:
!ls ./metastore_db/lucky_employees/*.parquet

./metastore_db/lucky_employees/part-00000-c81c8888-314f-4e44-abd8-8329050c21c6-c000.snappy.parquet
./metastore_db/lucky_employees/part-00001-c81c8888-314f-4e44-abd8-8329050c21c6-c000.snappy.parquet


In [194]:
# przykład złączania danych na różnych źródłach danych
# zapytanie SQL bezpośrednio na plikach - tutaj zapisanych wcześniej JSON-ach oraz parquet
query = """
SELECT ed.ID, ed.firstname, ed.lastname, ed.salary, lucky.rising, lucky.`salary after rising`
FROM json.`./employee_data.json/` as jtable 
join EMPLOYEE_DATA ed on jtable.ID=ed.ID 
join parquet.`./metastore_db/lucky_employees/` as lucky on ed.ID=lucky.ID
"""
df_from_json = spark.sql(query).show(10)

24/11/20 11:47:14 WARN ObjectStore: Failed to get database json, returning NoSuchObjectException
24/11/20 11:47:14 WARN ObjectStore: Failed to get database parquet, returning NoSuchObjectException


+----+----------+------------+--------+------+-------------------+
|  ID| firstname|    lastname|  salary|rising|salary after rising|
+----+----------+------------+--------+------+-------------------+
|  85|  Wojciech|      Wlotka| 8035.24|   10%|            8838.76|
| 989|Aleksandra|       Pysla| 8177.19|   10%|            8994.91|
|1727|      Adam|    Kowalski|  7317.8|   10%|            8049.58|
|1773|  Zbigniew|  Malinowski| 8484.68|   10%|            9333.15|
|1814| Katarzyna|  Malinowski| 6898.55|   10%|            7588.41|
|1894| Katarzyna|    Kowalski| 6235.94|   10%|            6859.53|
|2505|     Marek|      Wlotka| 6551.83|   10%|            7207.01|
|3723|     Marek|Mieczykowski| 8034.91|   10%|             8838.4|
|4502|Mieczysław|        Glut|10172.99|   10%|           11190.29|
|5146|     Marek|        Glut| 6995.49|   10%|            7695.04|
+----+----------+------------+--------+------+-------------------+
only showing top 10 rows



#### Dzielenie danych na wiaderka (ang. buckets) i partycje

Dzielenie danych na wiaderka jest rozwiązaniem, które stosowane jest do podziału danych na mniejsze części w sposób, który może przyspieszyć obliczenia poprzez zredukowanie liczby operacji przetasowania danych (ang. shuffle, a w kontekście Sparka mówimy o operacji exchange), które są bardzo kosztowne, gdyż wykonywane są często między węzłami (workerami).

In [128]:
# ten przykład pokazuje podział na 16 wiaderek danych bazując na podziale po kolumnie ID (tu używane jest hashowanie)
# dane posortowane są w każdym buckecie po kolumnie salary
# dane zapisywane są do hurtowni Hive, a informacje o zapisanych tam tabelach przechowywane są w
# Hive metastore (domyślnie jest do baza danych Derby)
df.write.bucketBy(16, 'ID').mode('overwrite').sortBy('salary').saveAsTable('employee_id_bucketed')

In [ ]:
!ls metastore_db/employee_salary_bucketed/*.parquet

In [129]:
spark.table('employee_id_bucketed').show(10)

+------+----------+------------+---+-------+
|    ID| firstname|    lastname|age| salary|
+------+----------+------------+---+-------+
|503945|   Wisława|    Barański| 54|3390.11|
|383781|   Wisława|    Barański| 39|4106.95|
|371700|     Marek|      Wlotka| 65|4155.71|
|110060|Aleksandra|Mieczykowski| 64|4213.95|
|260561|     Marek|        Glut| 47|4304.21|
|171761|   Wisława|Mieczykowski| 28| 4307.0|
| 97494|      Adam|        Glut| 30|4310.82|
|301272|Aleksandra|    Barański| 54|4343.63|
|248768|     Agata|        Glut| 42| 4352.4|
|   875|  Zbigniew|        Glut| 49|4385.59|
+------+----------+------------+---+-------+
only showing top 10 rows



In [81]:
# wypisanie tabeli
spark.catalog.listTables()

[Table(name='employee_bucketed', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='employees_partitioned_lastname', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='EMPLOYEE_DATA', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [84]:
# usunięcie tabeli
spark.sql('DROP TABLE employee_bucketed')

DataFrame[]

In [100]:
# jeżeli dane, z którymi pracujemy zawierają stosunkowo niewiele różnorodnych wartości w danych kolumnach
# lub filtrowanie i obliczenia często odbywają się na podgrupach danych to lepsze efekty uzyskamy
# poprzez wykorzystanie możliwości partycjonowania tych danych, które to partycjonowanie
# będzie również odzwierciedlone w fizycznej strukturze plików na dysku twardym w hurtowni danych

# zobaczmy przykład poniżej

df.write.partitionBy("lastname").mode('overwrite').saveAsTable("employees_partitioned_lastname")

In [75]:
# dobrym pomysłem jest też określenie ilości bucketów wynikających z danych w konkretnej kolumnie
# i wykorzystanie do podziału
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.bucketBy.html
buckets = spark.sql("select distinct firstname from EMPLOYEE_DATA").count()
buckets

10

In [103]:
# widok danych podzielonych na partycję z punktu widzenia systemu plików
!ls metastore_db/employees_partitioned_lastname

'lastname=Barański'		'lastname=Malinowski'	 'lastname=Wlotka'
'lastname=Brzęczyszczykiewicz'	'lastname=Mieczykowski'  'lastname=Wróblewski'
'lastname=Glut'			'lastname=Pysla'	  _SUCCESS
'lastname=Kowalski'		'lastname=Szczaw'


In [122]:
df.filter(df.lastname == 'Pysla').groupby('lastname').agg({'salary': 'avg'}).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[lastname#169], functions=[avg(salary#171)])
   +- Exchange hashpartitioning(lastname#169, 200), ENSURE_REQUIREMENTS, [plan_id=3316]
      +- HashAggregate(keys=[lastname#169], functions=[partial_avg(salary#171)])
         +- Filter (isnotnull(lastname#169) AND (lastname#169 = Pysla))
            +- FileScan csv [lastname#169,salary#171] Batched: false, DataFilters: [isnotnull(lastname#169), (lastname#169 = Pysla)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/work-dir/lab_06/employee_1m.csv], PartitionFilters: [], PushedFilters: [IsNotNull(lastname), EqualTo(lastname,Pysla)], ReadSchema: struct<lastname:string,salary:double>




In [123]:
%%time
df.filter(df.lastname == 'Pysla').groupby('lastname').agg({'salary': 'avg'}).show(10)

+--------+-----------------+
|lastname|      avg(salary)|
+--------+-----------------+
|   Pysla|7849.311716746816|
+--------+-----------------+

CPU times: user 10.7 ms, sys: 7.67 ms, total: 18.3 ms
Wall time: 2.68 s


In [124]:
spark.sql("select lastname, avg(salary) from employees_partitioned_lastname where lastname='Pysla' group by lastname").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[lastname#2348], functions=[avg(salary#2347)])
   +- Exchange hashpartitioning(lastname#2348, 200), ENSURE_REQUIREMENTS, [plan_id=3382]
      +- HashAggregate(keys=[lastname#2348], functions=[partial_avg(salary#2347)])
         +- FileScan parquet spark_catalog.default.employees_partitioned_lastname[salary#2347,lastname#2348] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/opt/spark/work-dir/lab_07/metastore_db/employees_partitioned_las..., PartitionFilters: [isnotnull(lastname#2348), (lastname#2348 = Pysla)], PushedFilters: [], ReadSchema: struct<salary:double>




In [125]:
%%time
spark.sql("select lastname, avg(salary) from employees_partitioned_lastname where lastname='Pysla' group by lastname").show(10)

+--------+-----------------+
|lastname|      avg(salary)|
+--------+-----------------+
|   Pysla|7849.311716746816|
+--------+-----------------+

CPU times: user 5.77 ms, sys: 0 ns, total: 5.77 ms
Wall time: 516 ms


Jak widać, operacja wykonała się szybciej.

In [7]:
spark.sparkContext.stop()

### Zadania

**Zadanie 1**  
Pamiętacie plik zamówienia.txt ?
Plik został umieszczony w folderze z labem w repozytorium.

Wczytaj ten plik za pomocą Sparka do dowolnego typu danych (RDD, Spark DataFrame) i dokonaj transformacji tak aby:
* naprawić problemy z kodowaniem znaków (replace?) w kolumnie Sprzedawca
* poprawić format danych w kolumnie Utarg
* dodać odpowiednie typy danych
* kolumna idZamowienia powinna być traktowana jako klucz (indeks)

**Zadanie 2**  
Po wykonaniu zadania 1, wykorzystaj przykłady z laboratorium i:
* 2.1 wykonaj wiaderkowanie danych i wykonaj dowolne zapytanie agregujące na tych danych vs. dane nie podzielone na wiaderka - porównaj czas
* 2.2 wykonaj partycjonowanie danych i zapisz je w formcie csv (wypróbuj partycjonowanie wg. kraju, nazwiska
* 2.3 wykonaj zapytanie agregujące z filtrowanie po kolumnie, której użyłeś/-aś do partycjonowania na danych oryginalnych oraz partycjonowanych i porównaj czas wykonania

**Zadanie 3**  
Z danych wygeneruj 4 różne podzbiory próbek (wiersze wybrane losowo) i dodaj nową kolumnę w każdym z nich, np. w jednym stwórz kolumnę month wyciągając tylko miesiąc z daty, w drugim wartość netto zamówienia (przyjmując, że vat to 23%), w kolejnym zamień nazwisko na wielkie litery, w kolejnym dodaj kolumnę waluta z wartością PLN.

Następnie zapisz każdy z tych zbiorów tak, że:
* zbiór pierwszy to będzie tymczasowa tabela in-memory Sparka
* zbiór drugi to plik(i) parquet
* zbiór trzeci to plik(i) csv
* zbiór czwarty to plik(i) json

Wykonaj zapytanie złączające jak w przykładzie pobierając dane bezpośrednio z plików i wyświetl idZamowienia, Kraj, Sprzedawcę, Datę, Utarg oraz 4 nowo utworzone kolumny.
